In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import json
import os
from datetime import timedelta


In [2]:
# Path to your uploaded CSV file
file_path = 'combined_data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Convert the 'Date' column to datetime format for easier handling
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'Date' column as the index of the DataFrame
df.set_index('Date', inplace=True)

# Display the first few rows of the DataFrame
print(df.head())


             AHLU    JMNA     MANB  COCHIN    JSTL     DBL     JUBI  GTLI  \
Date                                                                        
2012-07-02  51.80  13.260  163.875  528.15  67.950  251.75  122.580  8.20   
2012-07-03  52.00  13.185  163.875  528.15  69.515  251.75  122.465  9.40   
2012-07-04  52.00  13.040  163.875  528.15  72.620  251.75  122.155  9.15   
2012-07-05  51.90  13.005  163.875  528.15  72.275  251.75  124.120  9.15   
2012-07-06  50.65  12.920  163.875  528.15  71.585  251.75  124.305  9.40   

               SSOF     TCO  ...  UJJIVAN     SF   LICHF    SADE      BIL  \
Date                         ...                                            
2012-07-02  14.2500  95.665  ...   231.55  49.95  268.75  148.00  128.500   
2012-07-03  14.2500  95.705  ...   231.55  50.60  268.85  147.90  129.375   
2012-07-04  14.4375  95.460  ...   231.55  50.70  274.35  145.45  137.125   
2012-07-05  15.4125  96.955  ...   231.55  51.35  273.85  143.70  141.025  

In [3]:
import pandas as pd
import numpy as np
import json
from datetime import timedelta

# Load the ticker to name mapping from JSON
with open('./BacktestingData/mydata.json') as f:
    ticketToName = json.load(f)

# Date range for the backtest
start_date = pd.Timestamp('2012-07-02')
end_date = pd.Timestamp('2023-08-10')


# DataFrame to store NAV, stock names, and prices
dates = pd.date_range(start=start_date, end=end_date, freq='B')  # Business days between start and end date
columns = ['NAV'] + [f'Ticker_{i}' for i in range(1, 51)] + [f'Name_{i}' for i in range(1, 51)] + [f'Price_{i}' for i in range(1, 51)] + [f'PriceRatio_{i}' for i in range(1, 51)]
nav_data = pd.DataFrame(columns=columns)



In [4]:
# Set initial capital
capital = 10_00_000  # 10 lakhs, for example

# Initialize a dictionary to track stock purchases (date -> [ticker, number of shares])
stock_purchases = {}

# Trading Algorithm Parameters
lookback = 252  # Lookback period in days
holding_period = 21  # Holding period in days

# Initialize variables for tracking previous days
previous_day = None
previous_buy_day = None
previous_nav_update = None
date_set = {pd.Timestamp('2012-07-02')}


In [5]:
def get_date_from_NAV(current_day, lookback, date_set):
    if lookback == 7:
        prev_day = current_day - pd.DateOffset(weeks=1)
    elif lookback == 21:
        prev_day = current_day - pd.DateOffset(months=1)
    elif lookback == 63:
        prev_day = current_day - pd.DateOffset(months=3)
    elif lookback == 126:
        prev_day = current_day - pd.DateOffset(months=6)
    elif lookback == 252:
        prev_day = current_day - pd.DateOffset(years=1)

    # Ensure the previous day is a valid trading day in the date set
    while prev_day < current_day and prev_day not in date_set:
        prev_day += pd.DateOffset(days=1)
    if prev_day == current_day:
        print(f'Current: {current_day}')
    return prev_day

In [6]:
def largest_50(returns):
    # print(returns)
    top_stocks = []
    returns = returns.sort_values(ascending=False)
    # Drop NAN values
    if len(returns[returns != 0]) == 0:
        print(len(returns[returns < 0]))
        print(returns)
    
    returns = returns[returns != 0]
    # print(len(returns))
    return returns[:50].index

In [7]:
for current_day in pd.date_range(start=start_date, end=end_date):
    if current_day not in df.index:
        continue 

    date_set.add(current_day) 
    if current_day - start_date < timedelta(days=lookback):
        continue
    # Sell Logic
    if holding_period == 7:
        if previous_buy_day is not None and (current_day - previous_buy_day).days >= 7 or current_day.weekday == 0:
            new_cap = 0
            price_today = df.loc[current_day]
            # Sell all Stocks
            for stock, num_shares in stock_purchases[previous_buy_day]:
                new_cap += num_shares * price_today[stock]
            capital = new_cap
    elif holding_period == 21:
        if previous_buy_day is not None and (current_day.month != previous_buy_day.month or current_day.year != previous_buy_day.year):
            new_cap = 0
            price_today = df.loc[current_day]
            # Sell all Stocks
            for stock, num_shares in stock_purchases[previous_buy_day]:
                new_cap += num_shares * price_today[stock]
            capital = new_cap
        
    # Price Ratio Logic
    if previous_nav_update is not None and (current_day - previous_nav_update).days >= 7 or current_day.weekday == 0:
        price_today = df.loc[current_day]
        prev_prices = df.loc[previous_nav_update]
        # print(df.loc[current_day]['AHLU'])
        for i, stock in enumerate(stock_purchases[previous_buy_day]):
            # print(stock)
            nav_data.loc[previous_nav_update, f'PriceRatio_{i+1}'] = price_today[stock[0]] / prev_prices[stock[0]]
                        
    # Check if previous_buy_day was 7 or more days ago
    # Buy Logic

    if holding_period == 7:    
        if previous_buy_day is None or (current_day - previous_buy_day).days >= 7 or current_day.weekday == 0:
            # Calculate simple returns (price difference) between previous_buy_day and current_day
            prev_day = get_date_from_NAV(current_day, lookback, date_set)
            start_prices = df.loc[prev_day]
            end_prices = df.loc[current_day]
            returns = (end_prices - start_prices) / start_prices
            top_stocks = largest_50(returns)
            if len(top_stocks) == 0:
                print(current_day, prev_day)

            # Buy all 50 with equal proportion
            allocated_capital_per_stock = capital / len(top_stocks)
            stock_purchases[current_day] = []

            for stock in top_stocks:
                # Calculate the number of shares to buy for each stock
                num_shares = allocated_capital_per_stock / end_prices[stock]
                stock_purchases[current_day].append((stock, num_shares))
            previous_buy_day = current_day
    elif holding_period == 21:
        if previous_buy_day is None or (current_day.month != previous_buy_day.month or current_day.year != previous_buy_day.year):
            # Calculate simple returns (price difference) between previous_buy_day and current_day
            prev_day = get_date_from_NAV(current_day, lookback, date_set)
            start_prices = df.loc[prev_day]
            end_prices = df.loc[current_day]
            returns = (end_prices - start_prices) / start_prices
            top_stocks = largest_50(returns)
            if len(top_stocks) == 0:
                print(current_day, prev_day)

            # Buy all 50 with equal proportion
            allocated_capital_per_stock = capital / len(top_stocks)
            stock_purchases[current_day] = []

            for stock in top_stocks:
                # Calculate the number of shares to buy for each stock
                num_shares = allocated_capital_per_stock / end_prices[stock]
                stock_purchases[current_day].append((stock, num_shares))
            previous_buy_day = current_day
    

    
        
    # Nav Update Logic
    if previous_nav_update is None or (current_day - previous_nav_update).days >= 7 or current_day.weekday == 0:
        total_portfolio_value = 0
        price_today = df.loc[current_day]

        for i, (stock, num) in enumerate(stock_purchases[previous_buy_day]):
            stock_price = price_today[stock]
            stock_value = num * stock_price
            total_portfolio_value += stock_value
            nav_data.loc[current_day, f'Ticker_{i+1}'] = stock
            nav_data.loc[current_day, f'Name_{i+1}'] = ticketToName.get(stock + ' IN', 'Unknown')
            nav_data.loc[current_day, f'Price_{i+1}'] = stock_value
        nav_data.loc[current_day, 'NAV'] = total_portfolio_value
        previous_nav_update = current_day
        
            
        

In [ ]:
nav_data = nav_data.dropna(how='all')
# os.mkdir('NewResults', exist_ok=True)
os.chdir('NewResults')
name = 'Lookback_' + str(lookback) + '_Holding_' + str(holding_period)
nav_data.to_excel(name+'.xlsx')